# Bagging (with scikit-learn)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import sklearn.tree
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
import dtreeviz

## Our previous regression example

In [ ]:
x = np.linspace(0,10,100)

In [ ]:
np.random.seed(42)
noise = np.random.normal(0,1.5,100)

y = x + noise #0.3*(x-5)**3 + noise

In [ ]:
plt.plot(x,y,'ko')

## Straight-forward linear regression

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x.reshape(-1,1), y, random_state=42)

In [ ]:
lin_reg = sklearn.linear_model.LinearRegression()
lin_reg.fit(x_train,y_train)
print('R2 score: ',lin_reg.score(x_test, y_test))

In [ ]:
plt.plot(x_train,y_train,'ko')
plt.plot(x_test,y_test,'bo')
plt.plot([0,10],lin_reg.predict([[0],[10]]))

## BaggingRegressor with Linear Regression

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
# Be careful, by the way, about using Regressor vs Classifier (below we'll also use Classifiers)

bag_reg = BaggingRegressor(sklearn.linear_model.LinearRegression(), 
                           n_estimators=5,
                           max_samples=5, 
                           bootstrap=True, 
                           n_jobs=-1)
bag_reg.fit(x_train, y_train)
y_pred = bag_reg.predict(x_test)
print(bag_reg.__class__.__name__, sklearn.metrics.r2_score(y_test, y_pred))

plt.plot(x_train,y_train,'ko')
plt.plot(x_test,y_test,'bo')
plt.plot([0,10],bag_reg.predict([[0],[10]]), 'g')
plt.plot([0,10],lin_reg.predict([[0],[10]]), 'r')

Try tinkering around with the number of estimators and the number of samples.
* Note:  max_samples is the integer number of samples if you specify an integer, but it is the fraction of the total number of data points if you specify a float.

## Decision Tree

In [ ]:
tree_reg = sklearn.tree.DecisionTreeRegressor()
tree_reg.fit(x_train,y_train)
print('R2 score: ',tree_reg.score(x_test, y_test))

In [ ]:
plt.plot(x_train,y_train,'ko')
plt.plot(x_test,y_test,'bo')
xnew=np.linspace(0,10,1000).reshape(-1,1)
ynew=tree_reg.predict(xnew)
plt.plot(xnew,ynew)

## Random Forest

In [ ]:
rf_reg = sklearn.ensemble.RandomForestRegressor(n_jobs=-1,
                                                random_state=42)
rf_reg.fit(x_train,y_train)
print('R2 score: ',rf_reg.score(x_test, y_test))

In [ ]:
plt.plot(x_train,y_train,'ko')
plt.plot(x_test,y_test,'bo')
xnew=np.linspace(0,10,1000).reshape(-1,1)
ynew=rf_reg.predict(xnew)
plt.plot(xnew,ynew)

In [ ]:
rf_reg = sklearn.ensemble.RandomForestRegressor(max_depth=3,
                                                n_jobs=-1,
                                                random_state=42)
rf_reg.fit(x_train,y_train)
print('R2 score: ',rf_reg.score(x_test, y_test))

In [ ]:
plt.plot(x_train,y_train,'ko')
plt.plot(x_test,y_test,'bo')
xnew=np.linspace(0,10,1000).reshape(-1,1)
ynew=rf_reg.predict(xnew)
plt.plot(xnew,ynew)

# New Example:  Classification with Moons Dataset

In [ ]:
from sklearn.datasets import make_moons

In [ ]:
x, y = make_moons(n_samples=400, noise=0.3, random_state=42)

In [ ]:
sns.scatterplot(x=x[:,0], 
                y=x[:,1],
                hue=y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    random_state=0)

## Short review with VotingClassifier

In [ ]:
knn_clf = KNeighborsClassifier()
logreg_clf = LogisticRegression()
tree_clf = DecisionTreeClassifier()

In [ ]:
voting_clf = VotingClassifier(
    estimators=[('lr', logreg_clf), ('knn', knn_clf), ('tr', tree_clf)],
    voting='hard')

In [ ]:
voting_clf.fit(x_train, y_train)

In [ ]:
classifiers = [knn_clf, logreg_clf, tree_clf, voting_clf]

In [ ]:
for clf in classifiers:
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [ ]:
votingsoft_clf = VotingClassifier(
    estimators=[('lr', logreg_clf), ('knn', knn_clf), ('tr', tree_clf)],
    voting='soft')

votingsoft_clf.fit(x_train, y_train)

classifiers = [knn_clf, logreg_clf, tree_clf, voting_clf, votingsoft_clf]

for clf in classifiers:
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=500,
                            max_samples=100, 
                            bootstrap=True, 
                            n_jobs=-1)
bag_clf.fit(x_train, y_train)
y_pred = bag_clf.predict(x_test)
print(bag_clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [ ]:
# plot the decision boundary with the data

DecisionBoundaryDisplay.from_estimator(tree_clf, 
                                       x, 
                                       response_method="predict",
                                       cmap="RdBu", 
                                       alpha=0.5
)

sns.scatterplot(x=x[:,0], 
                y=x[:,1],
                hue=y,
                palette=['red','green'])

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
# plot the decision boundary with the data

DecisionBoundaryDisplay.from_estimator(bag_clf, 
                                       x, 
                                       response_method="predict",
                                       cmap="RdBu", 
                                       alpha=0.5
)

sns.scatterplot(x=x[:,0], 
                y=x[:,1],
                hue=y,
                palette=['red','green'])

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=500,
                            bootstrap=True, 
                            n_jobs=-1, 
                            oob_score=True)

bag_clf.fit(x_train, y_train)
bag_clf.oob_score_

In [ ]:
y_pred = bag_clf.predict(x_test)
accuracy_score(y_test, y_pred)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=500, 
                                 max_leaf_nodes=16, 
                                 n_jobs=-1,
                                 random_state=42)
rnd_clf.fit(x_train, y_train)

y_pred_rf = rnd_clf.predict(x_test)
accuracy_score(y_test, y_pred_rf)

In [ ]:
# equivalent bagging classifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(max_leaf_nodes=16),
                            n_estimators=500, 
                            max_samples=1.0, 
                            bootstrap=True, 
                            n_jobs=-1,
                            random_state=42)

In [ ]:
bag_clf.fit(x_train, y_train)
y_pred_rf = bag_clf.predict(x_test)
accuracy_score(y_test, y_pred_rf)

In [ ]:
classifiers = [knn_clf, logreg_clf, tree_clf, voting_clf, votingsoft_clf, rnd_clf]

In [ ]:
for clf in classifiers:
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [ ]:
# plot the decision boundary with the data

DecisionBoundaryDisplay.from_estimator(rnd_clf, 
                                       x, 
                                       response_method="predict",
                                       cmap="RdBu", 
                                       alpha=0.5
)

sns.scatterplot(x=x[:,0], 
                y=x[:,1],
                hue=y,
                palette=['red','green'])

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')